Retrieves images from "full_imgs" and stores each VOI separately in "train_imgs". Retrieves spreadsheet listing VOIs and stores them in text file. Requires data-retrieval to be run first.

In [65]:
import helper_fxns as hf
import transforms as tr
import importlib
import numpy as np
import os
import requests
import sys
import copy
import math
import pandas as pd
import random

In [73]:
with open("train_list.txt", "r") as f:
    vois = [x.split(',') for x in f.read().split("\n")]

In [74]:
voi_df = pd.DataFrame(vois, columns = ["Filename", "x1", "x2", "y1", "y2", "z1", "z2", "cls"]).dropna()
voi_df = voi_df.astype({"x1": int, "x2": int, "y1": int, "y2": int, "z1": int, "z2": int})

In [93]:
voi_df['dx'] = voi_df.apply(lambda row: row['x2'] - row['x1'], axis=1)
voi_df['dy'] = voi_df.apply(lambda row: row['y2'] - row['y1'], axis=1)
voi_df['dz'] = voi_df.apply(lambda row: row['z2'] - row['z1'], axis=1)

In [96]:
max(voi_df['dy'])

49

In [89]:
voi_df[voi_df["Filename"] == "12042703.npy"]

,Filename,x1,x2,y1,y2,z1,z2,cls,dx
13,12042703.npy,87,102,56,69,35,37,cyst,15
14,12042703.npy,90,102,71,85,30,32,cyst,12
15,12042703.npy,76,89,96,109,23,25,cyst,13
16,12042703.npy,64,73,127,135,12,14,cyst,9
17,12042703.npy,67,77,135,146,6,10,cyst,10
18,12042703.npy,65,76,138,148,0,3,cyst,11


In [83]:
def extract_voi(img, vois, final_size = [50,50,16], aug=False):
    voi_imgs = []
    classes = []
    
    if aug:
        crop_size = [math.ceil(x) for x in final_size*math.sqrt(2)]
    else:
        crop_size = final_size
    
    for _, voi in vois.iterrows():
        xpad = crop_size[0] - (voi['x2']-voi['x1'])
        ypad = crop_size[1] - (voi['y2']-voi['y1'])
        zpad = crop_size[2] - (voi['z2']-voi['z1'])
        
        side_padding = math.ceil(max(xpad, ypad, zpad) / 2)
        #print(side_padding)
        pad_img = np.pad(img, side_padding, 'constant')
        
        voi['x1'] += side_padding - math.ceil(xpad/2)
        voi['x2'] += side_padding + math.floor(xpad/2)
        voi['y1'] += side_padding - math.ceil(ypad/2)
        voi['y2'] += side_padding + math.floor(ypad/2)
        voi['z1'] += side_padding - math.ceil(zpad/2)
        voi['z2'] += side_padding + math.floor(zpad/2)
            
        if aug:
            return None
            base_img = pad_img[voi['x1']:voi['x2'], voi['y1']:voi['y2'], voi['z1']:voi['z2']]
            aug_imgs = aug_rotation(base_img, final_size)
            voi_imgs = voi_imgs + aug_imgs
            classes.append(voi['cls'] * len(aug_imgs))
            
        else:
            voi_imgs.append(pad_img[voi['x1']:voi['x2'], voi['y1']:voi['y2'], voi['z1']:voi['z2']])
            classes.append(voi['cls'])
    
    return voi_imgs, classes

In [84]:
for img_fn in os.listdir("full_imgs\\"):
    img = np.load("full_imgs\\"+img_fn)
    cropped_imgs, classes = extract_voi(img, voi_df[voi_df["Filename"] == img_fn])
    
    for cl in set(classes):
        subdir = "train_imgs\\"+cl
        if not os.path.exists(subdir):
            os.makedirs(subdir)
            
    for i in range(len(classes)):
        np.save("train_imgs\\"+classes[i]+"\\"+img_fn[:-4]+"_"+str(i), cropped_imgs[i])

KeyboardInterrupt: 

In [40]:
def aug_rotation(img, num_samples = 5):
    aug_imgs = [img]
    
    for _ in range(num_samples):
        angle = int(random.uniform(0, 365))
        aug_imgs.append(tr.rotate(img, angle))
        for roi in mrn_rois:
            aug_rois.append([mrn+"_r"+str(angle)+".npy"] + roi[1:])
    
    return aug_imgs

In [52]:
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in xrange(0, image.shape[0], stepSize):
        for x in xrange(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])